# Python modules for Statistical Analysis

The analytical techniques used by analysts to develop knowledge and information from data vary from the extremely simple to highly complex.  Most techniques that are available in commercially available packages are also to be found in Python, with the pandas and statsmodels library being central.  This note will briefly introduce these Python modules from the perspective of an analyst who is experienced with the standard statistical packages. 

This notebook is intended to complement an earlier notebook on the use of Python pandas to maintain databases. It will first load a database with artificial data.  This artificial data is created with interrelationships that will be revealed with subsequent analysis. However, the first step will be to perform the descriptive analysis to better understand the data.  Then there will be simple correlation analysis to scan for possible relationships between the variables in the database.  At this stage, the discussion will move from the pandas to another Python module, Statsmodels, which has a greater ability to analyze causality.

## Creating a Synthetic Database

The first step is to create a synthetic database.  It will embed various relationships and properties within the data that the analytical techniques will uncover.  The creation of the database will also provide a forum to showcase the analytic abiities of the packages available in Python. 

### Create a random variable

The NumPy and pandas libraries can be used to generate random variables. NumPy contains many random number generators.  The most basic is shown below where a vector of 5 numbers is produced. These numbers will be uniformly distributed between zero and one and change each time the program is run.

In [1]:
# Create random variable
import numpy as np
print(np.random.rand(5))

[ 0.19536934  0.19425416  0.33397571  0.31237523  0.47295437]


To freeze the random numbers, it is necessary to set the seed.

In [2]:
# fix the seed for the random numbers
np.random.seed(1)
print(np.random.rand(5))

[  4.17022005e-01   7.20324493e-01   1.14374817e-04   3.02332573e-01
   1.46755891e-01]


Now no matter how many times this notebook is run, the first element of the vector will always be 0.417.  

### Create a single random time series variable

For the purposes of this test a normally distributed Pandas time series will be created over a 50 year period, starting in 1966.  Essential it is the same process as above, except that a date is used to index the value of the series.

In [3]:
# Create a normally distributed random time series over a 50 year period starting in 1966
import pandas as pd
STARTYR = 1966
YRNO = 50
theyrper = pd.PeriodIndex(start=STARTYR, periods=YRNO,name='Year')
A1Y = pd.Series(np.random.randn(YRNO)+5,index=theyrper,name='A1Y')
print(A1Y['2000':'2005'])

Year
2000    5.275718
2001    3.909325
2002    4.390015
2003    5.306412
2004    6.691826
2005    4.252046
Freq: A-DEC, Name: A1Y, dtype: float64


There are a number of important points here.  First the Pandas module is used to create a time series object of annual data.  However, is important to note that this a random variable with a mean of around 5 and distributed normally.  The above was just a print of six of the observations, as all 50 would have been too much. The above variable, A1Y, is independent, with a mean of five and a standard deviation of roughly one. This was achived with the use of randn, which has a mean of zero, and adding 5 to it.  The default standard deviation is one. 

### Create remaining 6 variables

A2Y is created as a function of an error term A2E plus 0.5 of A1Y.  This creates a synthetic causal relationship that will be uncovered later on.  Note that it is necessary to add some randomness in the relationship.

In [4]:
# A2Y will be a function of A1Y plus an error term
A2E = pd.Series(np.random.randn(YRNO),index=theyrper,name='A2E')
A2Y = 0.5 * A1Y + A2E
print(A2Y['2000':'2005'])

Year
2000    2.848761
2001    2.376442
2002    2.776933
2003    2.242999
2004    5.642779
2005    3.814520
Freq: A-DEC, dtype: float64


A3Y will be a function of A1Y and A2Y.  

In [5]:
# A3Y will be a function of A1Y plus lag A2Y plus an error term
A3E = pd.Series(np.random.randn(YRNO),index=theyrper,name='A3E')
A3Y = 0.5 * A1Y + 2.0*A2Y.shift(1) + A3E
print(A3Y['2000':'2005'])

Year
2000     8.547386
2001     7.482411
2002     6.406746
2003     8.745019
2004     8.223193
2005    15.623496
Freq: A-DEC, dtype: float64


Here A3Y is created from two variables plus another error term.  However, in this case the lagged value of A2Y is used via the shift member function. This has the important implication of reducing the number of observations by one as it is impossible to calculate A3Y with a lagged variable missing.In the previous section A1Y, A2Y, A3Y were added to the database. In this section, three more series are created B1Y, B2Y, B3Y with similar yet different characteristics. This will allow the demonstration of SciPy's ability to be used in the analysis of panel data.

In [6]:
# Add a second dimension
B1Y = pd.Series(np.random.randn(YRNO)*5+50,index=theyrper,name='B1Y')
B2E = pd.Series(np.random.randn(YRNO),index=theyrper,name='B2E')
B2Y = 2.0 * B1Y + B2E*5
B3E = pd.Series(np.random.randn(YRNO),index=theyrper,name='B3E')
B3Y = 0.5 * B1Y + 2.0*B2Y.shift(1) - A3Y + B3E*5


This creates three new variables with similar relationships to above.  One new twist is the that A3Y has a negative impact on B3Y.

### Load Six Time Series into Dataset

We now have six data series that will now loaded into a single two dimensional DataFrame, theDF. Two create this DataFrame an index is created that allows for two layers of indexing.  One layer of indexing, designates the synthetic regions in this example as 'A' and 'B'.  The second layer, designates the industry as either one of three: 'Ind1','Ind2' and 'Ind3'.

The DataFrame Ydf is created using a list of two lists designating the columns.  Then the time series are allocated to Ydf using a dictionary like syntax but with two elements, representing industry and region.  The dimensions of this database are 50 by 6, which are the years by the number variables. However, as we will see, the hierarchic indexing allows the analysis to resemble what is possible with a panel dataset. 


In [7]:
# Create Tags for the two Regions A and B
RegionTag = [ 'A' for letno in range(3) ] + [ 'B' for letno in range(3) ]
IndTag = ['Ind1','Ind2','Ind3'] *2
 
Ydf = pd.DataFrame(columns=[RegionTag,IndTag])
Ydf.columns.names=['Region','Industry']
Ydf['A','Ind1'] = A1Y
Ydf['A','Ind2'] = A2Y
Ydf['A','Ind3'] = A3Y
Ydf['B','Ind1'] = B1Y
Ydf['B','Ind2'] = B2Y
Ydf['B','Ind3'] = B3Y
del A1Y,A2Y,A3Y,B1Y,B2Y,B3Y
print("The dimensions of Ydf are {}".format(Ydf.shape))


The dimensions of Ydf are (50, 6)


In [8]:
# Validate counts of observations
Ydf.count()

Region  Industry
A       Ind1        50
        Ind2        50
        Ind3        49
B       Ind1        50
        Ind2        50
        Ind3        49
dtype: int64

As can be seen by the count table above, the data is loaded in the pandas dataset.  Note that only 49 observations were available for both Ind3's as they were a function of the previous years value of their respective Ind2's.  As these did not exist in first year, 1966, the values are missing.  

In [9]:
# Show missing first observations for variables function of lags
Ydf.head()

Region           A                              B                        
Industry      Ind1      Ind2      Ind3       Ind1        Ind2        Ind3
Year                                                                     
1966      3.894065  3.195332       NaN  43.786271   93.411727         NaN
1967      3.345485  0.915068  7.722578  57.997415  109.932382  204.240403
1968      2.636531  1.906560  3.635996  62.372097  113.960668  246.090013
1969      6.135345  3.414532  8.619865  48.338376   92.183413  249.007069
1970      3.982986  3.358526  8.889527  53.885733   99.237776  195.676484

Both the above view of observation counts and actual values through time are organized by both Region and Industry. Although in creating Ydf the pandas panel option was not used, this use of a multilevel or heirarchic index gives credence to the desription of pandas as a panel dataset.  

In [10]:
# retrieve individual time series from 
Ydf['A','Ind1']['2000':'2005']

Year
2000    5.275718
2001    3.909325
2002    4.390015
2003    5.306412
2004    6.691826
2005    4.252046
Freq: A-DEC, Name: (A, Ind1), dtype: float64

Here there is a retrieval region, industry and time. The above example duplicates the values obtained above for A1Y. In a sense it does demonstrates the panel nature of pandas.

## Descriptive Statistics

Before any analysis can begin, it is always useful to calculate descriptive statistics.  The analysis at this stage will use the tools available in pandas.  However, in the final section on causality, the Statsmodels library will be used.

In [11]:
# Look at means
Ydf.mean()

Region  Industry
A       Ind1          4.993288
        Ind2          2.743396
        Ind3          8.082713
B       Ind1         51.225903
        Ind2        101.653703
        Ind3        221.909325
dtype: float64

In [12]:
Ydf.describe()


Region            A                                B                        
Industry       Ind1       Ind2       Ind3       Ind1        Ind2        Ind3
count     50.000000  50.000000  49.000000  50.000000   50.000000   49.000000
mean       4.993288   2.743396   8.082713  51.225903  101.653703  221.909325
std        0.987552   1.008448   2.276365   4.453299    9.249725   20.149059
min        2.636531   0.528250   3.635996  40.810122   85.231523  187.686389
25%        4.175472   2.023663   7.038333  48.496204   95.881541  205.552698
50%        5.063024   2.806173   8.135011  51.001967  100.819258  222.492180
75%        5.651554   3.400257   9.299919  53.863628  107.089306  234.760764
max        7.042029   5.642779  15.623496  62.372097  128.483829  276.977423

## Correlation Analysis

If the analysts is happy with the initial descriptive statistics, then they may move on to an examination of the basic correlations.  This can be still effectively done in pandas, although at this point some analysts will want to move to another module, such a Statmodels, which we will see in the next section.

### Correlation Between Individual Time Series

Correlation can exists between time series as well as between observations in the time series itself.  With pandas it is easy to examine the correlations between the variables themselves.  If we remember that A2Y was made a function of A1Y

In [13]:
#Examine correlation between two time series that are related by construction
Ydf['A','Ind1'].corr(Ydf['A','Ind2'])

0.5319552897630998

Here there is a very definite correlation that is in line with construction of the variable.  It is useful to compare with A1Y and B2Y, who are not correlated by construction.

In [14]:
# Examine the correlation between two synthetic time series whose construction was independent
Ydf['A','Ind1'].corr(Ydf['B','Ind1'])

-0.16205417914269171

Here, although the correlation is not zero, it is within the relm of what may have occured by chance.

### Correlation within the matrix

With a small database of only six time series variables, pandas can provide a quick table for displaying the correlations within the database.

In [15]:
# Show database
Ydf.corr()

Region                  A                             B                    
Industry             Ind1      Ind2      Ind3      Ind1      Ind2      Ind3
Region Industry                                                            
A      Ind1      1.000000  0.531955  0.184400 -0.162054 -0.166880  0.012834
       Ind2      0.531955  1.000000  0.221364 -0.153066 -0.169759 -0.074736
       Ind3      0.184400  0.221364  1.000000  0.112122  0.123070 -0.137632
B      Ind1     -0.162054 -0.153066  0.112122  1.000000  0.844512 -0.059774
       Ind2     -0.166880 -0.169759  0.123070  0.844512  1.000000 -0.106321
       Ind3      0.012834 -0.074736 -0.137632 -0.059774 -0.106321  1.000000

This table is similar to what would be produced in most statistics packages. The correlation between A1Y and A2Y is 0.53, as was shown with the individual examinations.  Similarly, the low correlation between A1Y and A2Y is shown as well.

This form of analysis shows it value when the higher correlations jump out, such as B1Y and B2Y.  However, if the database were substantially larger, then it would be more practical to use other packages than Pandas.

## Causal Analysis

The purpose of this section is to demonstrate that many of the tools that applied analysts use in the business world exist within the Python ecosystem.  However, it is important to note that causality is a big issue, and this note will not resolve it. It will only display some of the rudimentary tools. 

By construction A1Y had a positive influence on A2Y.  This can be confirmed with regression analysis. It is possible to do some of the regression in analysis in pandas. However, at this point it is useful to demonstrate that the analysis can be done in the StatModel package that is focussed on the application of statistical techniques, rather than using the capabilities that exist in pandas.

### Simple Causal Relations

pandas data is easily analysed with the module Statsmodels. First the data needs to be extracted to a timeseries variable, then it is included in a standard regression formulation.  Note, that even this step may been unnecessary with other means of constructing Ydf.

The regression is run below with the fit() command, which returns the results as an object.

In [16]:
# Run regression of A2Y against A1Y.
import statsmodels.formula.api as smf
# extract time series for regression
A2Y = Ydf['A','Ind2']
A1Y = Ydf['A','Ind1']
mod_A2Y_A1Y = smf.ols(formula='A2Y ~ A1Y' , data=Ydf).fit()
print(mod_A2Y_A1Y.summary())

                            OLS Regression Results                            
Dep. Variable:                    A2Y   R-squared:                       0.283
Model:                            OLS   Adj. R-squared:                  0.268
Method:                 Least Squares   F-statistic:                     18.94
Date:                Mon, 04 Sep 2017   Prob (F-statistic):           7.02e-05
Time:                        18:31:07   Log-Likelihood:                -62.546
No. Observations:                  50   AIC:                             129.1
Df Residuals:                      48   BIC:                             132.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0310      0.635      0.049      0.9

Above the Python statsmodel module produces regression results similar to what would be obtained with a commercial package.  However, as the module is open source, a dedicated analyst would have the freedom to modify the contents of the output. There are some key points to note:
* The interface between the pandas module and statsmodels is reasonable.  
* The command interface is very similar to R
* The estimated coefficient of 0.54 is reasonable close to the "true" coefficient used to generate the data
    * The significance test suggests that it is unlikely the value is zero
* The "true" constant was 0 and the estimated value is not significantly different than zero

The above regression results was based on two variables known to be related. In the second case, A1Y and B1Y were not correlated.  This is confirmed below:

In [17]:
# regression where correlation is spurious
# recover time series for regression
B1Y = Ydf['B','Ind1']
A1Y = Ydf['A','Ind1']
mod_A1Y_B1Y = smf.ols(formula='A1Y ~ B1Y', data=Ydf).fit()
print(mod_A1Y_B1Y.summary())

                            OLS Regression Results                            
Dep. Variable:                    A1Y   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.295
Date:                Mon, 04 Sep 2017   Prob (F-statistic):              0.261
Time:                        18:31:07   Log-Likelihood:                -69.150
No. Observations:                  50   AIC:                             142.3
Df Residuals:                      48   BIC:                             146.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.8342      1.624      4.208      0.0

As expected the coefficient on B1Y was virtually zero.  This is expected by construction with an intercept of close to the synthetic value.  The strongest causal relation that appeared in the correlation matrix was between B1Y and B2Y.  As seen below, a tighter R-squared and test of significance.

In [18]:
# regression with very strong relation
B2Y = Ydf['B','Ind2']
B1Y = Ydf['B','Ind1']
mod_B2Y_B1Y = smf.ols(formula='B2Y ~ B1Y', data=Ydf).fit()
print(mod_B2Y_B1Y.summary())

                            OLS Regression Results                            
Dep. Variable:                    B2Y   R-squared:                       0.713
Model:                            OLS   Adj. R-squared:                  0.707
Method:                 Least Squares   F-statistic:                     119.4
Date:                Mon, 04 Sep 2017   Prob (F-statistic):           1.29e-14
Time:                        18:31:07   Log-Likelihood:                -150.45
No. Observations:                  50   AIC:                             304.9
Df Residuals:                      48   BIC:                             308.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     11.7987      8.255      1.429      0.1

Here we have a far tighter fit as evidenced by the higher R-square and test statistics.

### Multiple Regression

The above regression only examined the relationship between pairs of variables.  However, more complex relationships can be examined.  Below we estimate the relationships between B3Y and the variables that went into its construction.  Here three exogenous variables contribute to its value.  Of particular note is B2Y where the shift member function is used to allow for the lag effect.

In [19]:
# multiple regression with a lagged variable
B3Y = Ydf['B','Ind3']
B1Y = Ydf['B','Ind1']
B2Y = Ydf['B','Ind2']
A3Y = Ydf['A','Ind3']
mod_B3Y = smf.ols(formula='B3Y ~ B1Y + B2Y.shift(1) + A3Y', data=Ydf).fit()
print(mod_B3Y.summary())

                            OLS Regression Results                            
Dep. Variable:                    B3Y   R-squared:                       0.945
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                     257.4
Date:                Mon, 04 Sep 2017   Prob (F-statistic):           2.48e-28
Time:                        18:31:07   Log-Likelihood:                -145.15
No. Observations:                  49   AIC:                             298.3
Df Residuals:                      45   BIC:                             305.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -4.8996     12.350     -0.397   

## Concluding Notes

The above demonstrates that Python is a viable environment to do statistical basic analysis.  It may be many years before the thousands of models available in packages such as R, are available to Python users.  Still, the basic models are there and in an environment that is relatively easy to use if the investment in Python has already been made.